In [21]:
import requests
import json
import pandas as pd

In [25]:
with open("../apikey", "r") as f:
    apikey = f.read()

# Steam API

The official Steam API is largely undocumented. Reference from: https://github.com/Revadike/InternalSteamWebAPI/wiki/Get-App-Details

Let's start with the GetAppList request

In [31]:
url = f"https://api.steampowered.com/IStoreService/GetAppList/v1/?key={apikey}"

response = requests.get(url).json()
response['response'].keys()


dict_keys(['apps', 'have_more_results', 'last_appid'])

The response returns three objects: a list of apps, a boolean indicating whether there are more results, and the last app id returned (to use as a start point for the next request)

In [29]:
applist = pd.DataFrame(data['response']['apps']).set_index('appid')
applist.head(10)

,name,last_modified,price_change_number
appid,,,
10,Counter-Strike,1666823513,22856550
20,Team Fortress Classic,1579634708,22856550
30,Day of Defeat,1512413490,22856550
40,Deathmatch Classic,1568752159,22856550
50,Half-Life: Opposing Force,1579628243,22856550
60,Ricochet,1599518374,22856550
70,Half-Life,1700269108,23366843
80,Counter-Strike: Condition Zero,1602535977,22856550
130,Half-Life: Blue Shift,1579629868,22856550


Now let's see what details we can get on individual games

In [40]:
appid = applist.index[0]
url = f"https://store.steampowered.com/api/appdetails?appids={appid}"

response = requests.get(url).json()
print(response[str(appid)].keys())

game_data = response[str(appid)]["data"]

dict_keys(['success', 'data'])


In [41]:
game_data.keys()

dict_keys(['type', 'name', 'steam_appid', 'required_age', 'is_free', 'detailed_description', 'about_the_game', 'short_description', 'supported_languages', 'header_image', 'capsule_image', 'capsule_imagev5', 'website', 'pc_requirements', 'mac_requirements', 'linux_requirements', 'developers', 'publishers', 'price_overview', 'packages', 'package_groups', 'platforms', 'metacritic', 'categories', 'genres', 'screenshots', 'recommendations', 'release_date', 'support_info', 'background', 'background_raw', 'content_descriptors', 'ratings'])

# SteamSpy API

SteamSpy is a third party service that collects stats on Steam games. This includes number of owners for games, estimated using a sampling methodology.

Reference:
* https://steamspy.com/about
* https://steamspy.com/api.php

## All games list
We'll start with the all request which returns 1000 games at a time, sorted by number of owners

In [43]:
url = "https://steamspy.com/api.php?request=all&page=0"
response = requests.get(url).json()
respoen